In [287]:
import pandas as pd
import numpy as np
from pathlib import Path

In [288]:

boroughs = ["Bronx", "Brooklyn", "Manhattan", "Queens", "StatenIsland"]
# store data_frames for each borough
borough_frames={}
# read .csv files for each borough into dataframe, append year column
for borough in boroughs:
    borough_frames[borough] = []
    for Year in range(2003,2019):
        file_name= f"{year}_{borough}.csv"
        file_path = Path(borough,borough,file_name)
        year_frame = pd.read_csv(file_path, index_col=0)
        year_frame = year_frame.assign(Year=Year)
        borough_frames[borough].append(year_frame)

In [289]:
#borough_frames["Bronx"][0]["Building Class Category"].unique()
borough_frames["Bronx"][5].head()

,Neighborhood,Building Class Category,Tax Class At Present,Block,Lot,Ease-Ment,Building Class At Present,Address,Apartment Number,Zip Code,...,Commercial Units,Total Units,Land Square Feet,Gross Square Feet,Year Built,Tax Class At Time Of Sale,Building Class At Time Of Sale,Sale Price,Sale Date,Year
Borough,,,,,,,,,,,,,,,,,,,,,
2,BATHGATE,01 ONE FAMILY HOMES,1,2917,15,,A1,1848 WASHINGTON AVENUE,,10457,...,0,1,2943,2128,1920,1,A1,130000,2003-05-27 00:00:00,2008
2,BATHGATE,01 ONE FAMILY HOMES,1,3028,25,,A5,412 EAST 179 STREET,,10457,...,0,1,1842,2048,1901,1,A1,204000,2003-04-07 00:00:00,2008
2,BATHGATE,01 ONE FAMILY HOMES,1,3030,55,,A5,410 EAST 182 STREET,,10457,...,0,1,1330,1460,1899,1,A1,235000,2003-07-24 00:00:00,2008
2,BATHGATE,01 ONE FAMILY HOMES,1,3035,2,,S1,441 EAST 178 STREET,,10457,...,1,2,1287,2378,1899,1,S1,125500,2003-05-02 00:00:00,2008
2,BATHGATE,01 ONE FAMILY HOMES,1,3035,52,,A1,461 EAST 178 STREET,,10457,...,0,1,1782,1548,1899,1,A1,215000,2003-05-14 00:00:00,2008


In [290]:
#concatenate dataframes for each year into one dataframe/borough; 
for k, v in borough_frames.items():
    borough_frames[k] = pd.concat(v)
    

In [291]:
#drop_columns = ["Tax Class At Present", "Block", "Lot", "Ease-Ment", "Building Class At Present", 
#"Commercial Units", "Total Units", "Year Built", "Tax Class At Time Of Sale", "Building Class At Time Of Sale"]
select_columns = ["Borough", "Neighborhood", "Building Class Category", "Address", "Zip Code", "Residential Units", 
                  "Land Square Feet", "Gross Square Feet", 
                  "Year", "Sale Price", "Sale Date"]
selected_col = {}
# Drop unnecessary columns; also replace spaces in col names with underscore
for k,v in borough_frames.items():
    selected_col[k] = v.filter(select_columns, axis=1)
    selected_col[k].columns = selected_col[k].columns.str.replace(' ','_')
    print(k, selected_col[k].shape)
    

Bronx (170608, 10)
Brooklyn (539280, 10)
Manhattan (355344, 10)
Queens (626816, 10)
StatenIsland (202720, 10)


In [270]:
# get building classes for all dataframes in one array
building_classes=[]
for k,v in selected_col.items():
    building_classes.append(v["Building_Class_Category"].unique())
building_classes = np.concatenate(building_classes)

np.unique(building_classes)


array(['01  ONE FAMILY HOMES                        ',
       '02  TWO FAMILY HOMES                        ',
       '03  THREE FAMILY HOMES                      ',
       '04  TAX CLASS 1 CONDOS                      ',
       '05  TAX CLASS 1 VACANT LAND                 ',
       '06  TAX CLASS 1 - OTHER                     ',
       '07  RENTALS - WALKUP APARTMENTS             ',
       '08  RENTALS - ELEVATOR APARTMENTS           ',
       '09  COOPS - WALKUP APARTMENTS               ',
       '10  COOPS - ELEVATOR APARTMENTS             ',
       '11  SPECIAL CONDO BILLING LOTS              ',
       '12  CONDOS - WALKUP APARTMENTS              ',
       '13  CONDOS - ELEVATOR APARTMENTS            ',
       '14  RENTALS - 4-10 UNIT                     ',
       '15  CONDOS - 2-10 UNIT RESIDENTIAL          ',
       '16  CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT ',
       '17  CONDOPS                                 ',
       '18  TAX CLASS 3 - UTILITY PROPERTIES        ',
       '21

In [271]:
#select only residential classes; ?? Tax class 06; should similar categories be 
#collapsed for final analysis/presentation;
#mixed residential and commercial; buildings with multiple units sold as a 
#whole (i.e. will drive average for that type upward);
#should # of residential and commercial units be used to balance
residential_classes = ["01", "02", "03", "04", "06", "07", "08", "09", "10", 
                         "12", "13", "14", "15", "16", "17", "23", "28"]

#data_frame with selected columns and only (?) residential properties
#selected_col.Building_Class_Category
#might need to add unique index

res_df = {}

for k,v in selected_col.items():
    res_df[k] = v[v["Building_Class_Category"].str[:2].isin(residential_classes)]

In [272]:
# verify that only target classes remain
building_classes=[]
for k,v in res_df.items():
    building_classes.append(v["Building_Class_Category"].unique())
building_classes = np.concatenate(building_classes)

np.unique(building_classes)



array(['01  ONE FAMILY HOMES                        ',
       '02  TWO FAMILY HOMES                        ',
       '03  THREE FAMILY HOMES                      ',
       '04  TAX CLASS 1 CONDOS                      ',
       '06  TAX CLASS 1 - OTHER                     ',
       '07  RENTALS - WALKUP APARTMENTS             ',
       '08  RENTALS - ELEVATOR APARTMENTS           ',
       '09  COOPS - WALKUP APARTMENTS               ',
       '10  COOPS - ELEVATOR APARTMENTS             ',
       '12  CONDOS - WALKUP APARTMENTS              ',
       '13  CONDOS - ELEVATOR APARTMENTS            ',
       '14  RENTALS - 4-10 UNIT                     ',
       '15  CONDOS - 2-10 UNIT RESIDENTIAL          ',
       '16  CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT ',
       '17  CONDOPS                                 ',
       '23  LOFT BUILDINGS                          ',
       '28  COMMERCIAL CONDOS                       '], dtype=object)

,Neighborhood,Building_Class_Category,Address,Zip_Code,Residential_Units,Land_Square_Feet,Gross_Square_Feet,Year,Sale_Price,Sale_Date
Borough,,,,,,,,,,
2,BATHGATE,01 ONE FAMILY HOMES,1848 WASHINGTON AVENUE,10457,1,2943,2128,2003,130000,2003-05-27 00:00:00
2,BATHGATE,01 ONE FAMILY HOMES,412 EAST 179 STREET,10457,1,1842,2048,2003,204000,2003-04-07 00:00:00
2,BATHGATE,01 ONE FAMILY HOMES,410 EAST 182 STREET,10457,1,1330,1460,2003,235000,2003-07-24 00:00:00
2,BATHGATE,01 ONE FAMILY HOMES,441 EAST 178 STREET,10457,1,1287,2378,2003,125500,2003-05-02 00:00:00
2,BATHGATE,01 ONE FAMILY HOMES,461 EAST 178 STREET,10457,1,1782,1548,2003,215000,2003-05-14 00:00:00


In [274]:
# filter based on price
for k,v in res_df.items():
    res_df[k] = v[v.Sale_Price > 100]

In [286]:
for k,v in res_df.items():
    print(k, v.shape)

Bronx (99776, 10)
Brooklyn (297920, 10)
Manhattan (288784, 10)
Queens (393488, 10)
StatenIsland (128048, 10)
